In [ ]:
!pip3 install pyspark

In [ ]:
!export PYSPARK_SUBMIT_ARGS="--master local[2] pyspark-shell"

In [168]:
import pyspark
from pyspark.sql import SparkSession
from urllib.parse import quote
import pymongo
from pyspark import SparkContext,SparkConf
from pyspark.sql.functions import col
from pyspark.sql.functions import isnull, when, count, col, isnan, col, isnan, when, trim
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.feature import MinMaxScaler

In [2]:
pyspark.__version__

'3.2.0'

In [3]:
spark = SparkSession\
    .builder\
    .master('local')\
    .config('spark.mongodb.input.uri', 'mongodb+srv://ajinkya30:'+quote('Ajinkya@30')+'@clusterstock.jw7kb.mongodb.net/DB_Stock.Record_Stock?retryWrites=true&w=majority')\
    .config('spark.mongodb.output.uri', 'mongodb+srv://ajinkya30:'+quote('Ajinkya@30')+'@clusterstock.jw7kb.mongodb.net/DB_Stock.Record_Stock?retryWrites=true&w=majority')\
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1')\
    .getOrCreate()

In [4]:
spark

In [103]:
df = spark.read\
    .format("com.mongodb.spark.sql.DefaultSource")\
    .option("database","DB_Stock")\
    .option("collection", "Record_Stock")\
    .load()

In [62]:
df

DataFrame[Name: string, _id: struct<oid:string>, close: double, date: string, diff: double, high: double, low: double, open: double, volume: int]

In [63]:
df.show(5)

+----+--------------------+-----+----------+-------------------+-----+-----+-----+--------+
|Name|                 _id|close|      date|               diff| high|  low| open|  volume|
+----+--------------------+-----+----------+-------------------+-----+-----+-----+--------+
| AAL|{6196c127a87a68a4...|14.75|2013-02-08|0.48999999999999844|15.12|14.63|15.07| 8407500|
| AAL|{6196c127a87a68a4...|14.46|2013-02-11|               0.75|15.01|14.26|14.89| 8882000|
| AAL|{6196c127a87a68a4...|14.27|2013-02-12|0.41000000000000014|14.51| 14.1|14.45| 8126000|
| AAL|{6196c127a87a68a4...|14.66|2013-02-13| 0.6899999999999995|14.94|14.25| 14.3|10259500|
| AAL|{6196c127a87a68a4...|13.99|2013-02-14| 1.8000000000000007|14.96|13.16|14.94|31879900|
+----+--------------------+-----+----------+-------------------+-----+-----+-----+--------+
only showing top 5 rows



In [64]:
data = df.toPandas()

In [65]:
data

,Name,_id,close,date,diff,high,low,open,volume
0,AAL,"(6196c127a87a68a43a04bec0,)",14.75,2013-02-08,0.49,15.12,14.63,15.07,8407500
1,AAL,"(6196c127a87a68a43a04bec1,)",14.46,2013-02-11,0.75,15.01,14.26,14.89,8882000
2,AAL,"(6196c127a87a68a43a04bec2,)",14.27,2013-02-12,0.41,14.51,14.10,14.45,8126000
3,AAL,"(6196c127a87a68a43a04bec3,)",14.66,2013-02-13,0.69,14.94,14.25,14.30,10259500
4,AAL,"(6196c127a87a68a43a04bec4,)",13.99,2013-02-14,1.80,14.96,13.16,14.94,31879900
...,...,...,...,...,...,...,...,...,...
619035,ZTS,"(6196c12da87a68a43a0e30db,)",77.82,2018-02-01,1.58,78.27,76.69,76.84,2982259
619036,ZTS,"(6196c12da87a68a43a0e30dc,)",76.78,2018-02-02,1.39,78.12,76.73,77.53,2595187
619037,ZTS,"(6196c12da87a68a43a0e30dd,)",73.83,2018-02-05,3.74,76.92,73.18,76.64,2962031
619038,ZTS,"(6196c12da87a68a43a0e30de,)",73.27,2018-02-06,2.43,74.56,72.13,72.74,4924323


In [9]:
df

DataFrame[Name: string, _id: struct<oid:string>, close: double, date: string, diff: double, high: double, low: double, open: double, volume: int]

In [66]:
df.count()

619040

In [15]:
df.dtypes

[('Name', 'string'),
 ('_id', 'struct<oid:string>'),
 ('close', 'double'),
 ('date', 'string'),
 ('diff', 'double'),
 ('high', 'double'),
 ('low', 'double'),
 ('open', 'double'),
 ('volume', 'int')]

See the decsriptive statistics

In [67]:
df.describe().toPandas()

,summary,Name,close,date,diff,high,low,open,volume
0,count,619040,619040,619040,619040,619040,619040,619040,619040
1,mean,None,83.04376276476455,None,NaN,NaN,NaN,NaN,4321823.395568945
2,stddev,None,97.38974800165754,None,NaN,NaN,NaN,NaN,8693609.511967564
3,min,A,1.59,2013-02-08,-0.25500000000000256,1.69,1.5,1.62,0
4,max,ZTS,2049.0,2018-02-07,NaN,NaN,NaN,NaN,618237630


Convert the date column to date type

In [104]:
dataset = df.select(col('date').cast('date'),col('Name'),col('open'),col('close'),col('high'),col('low'),col('volume'),col('diff'))

In [105]:
dataset.show()

+----------+----+-----+-----+-----+-----+--------+-------------------+
|      date|Name| open|close| high|  low|  volume|               diff|
+----------+----+-----+-----+-----+-----+--------+-------------------+
|2013-02-08| AAL|15.07|14.75|15.12|14.63| 8407500|0.48999999999999844|
|2013-02-11| AAL|14.89|14.46|15.01|14.26| 8882000|               0.75|
|2013-02-12| AAL|14.45|14.27|14.51| 14.1| 8126000|0.41000000000000014|
|2013-02-13| AAL| 14.3|14.66|14.94|14.25|10259500| 0.6899999999999995|
|2013-02-14| AAL|14.94|13.99|14.96|13.16|31879900| 1.8000000000000007|
|2013-02-15| AAL|13.93| 14.5|14.61|13.93|15628000| 0.6799999999999997|
|2013-02-19| AAL|14.33|14.26|14.56|14.08|11354400| 0.4800000000000004|
|2013-02-20| AAL|14.17|13.33|14.26|13.15|14725200| 1.1099999999999994|
|2013-02-21| AAL|13.62|13.37|13.95| 12.9|11922100|  1.049999999999999|
|2013-02-22| AAL|13.57|13.57| 13.6|13.21| 6071400| 0.3899999999999988|
|2013-02-25| AAL| 13.6|13.02|13.76| 13.0| 7186400| 0.7599999999999998|
|2013-

To check null values

In [73]:
dataset.select([count(when(isnull(c), c)).alias(c) for c in dataset]).show()

+----+----+----+-----+----+---+------+----+
|date|Name|open|close|high|low|volume|diff|
+----+----+----+-----+----+---+------+----+
|   0|   0|   0|    0|   0|  0|     0|   0|
+----+----+----+-----+----+---+------+----+



In [92]:
dataset_1 = dataset.select(col('open'),col('close'),col('high'),col('low'),col('volume'),col('diff'))

To check the NAN values

In [81]:
dataset.select([count(when(isnan(c), c)).alias(c) for c in dataset_1.columns]).show()

+----+-----+----+---+------+----+
|open|close|high|low|volume|diff|
+----+-----+----+---+------+----+
|  11|    0|   8|  8|     0|   8|
+----+-----+----+---+------+----+



Encoding the name column

In [106]:
dataset = StringIndexer(
    inputCol='Name', 
    outputCol='Stock_Abbrv', 
    handleInvalid='keep').fit(dataset).transform(dataset)

In [107]:
dataset.show()

+----------+----+-----+-----+-----+-----+--------+-------------------+-----------+
|      date|Name| open|close| high|  low|  volume|               diff|Stock_Abbrv|
+----------+----+-----+-----+-----+-----+--------+-------------------+-----------+
|2013-02-08| AAL|15.07|14.75|15.12|14.63| 8407500|0.48999999999999844|        1.0|
|2013-02-11| AAL|14.89|14.46|15.01|14.26| 8882000|               0.75|        1.0|
|2013-02-12| AAL|14.45|14.27|14.51| 14.1| 8126000|0.41000000000000014|        1.0|
|2013-02-13| AAL| 14.3|14.66|14.94|14.25|10259500| 0.6899999999999995|        1.0|
|2013-02-14| AAL|14.94|13.99|14.96|13.16|31879900| 1.8000000000000007|        1.0|
|2013-02-15| AAL|13.93| 14.5|14.61|13.93|15628000| 0.6799999999999997|        1.0|
|2013-02-19| AAL|14.33|14.26|14.56|14.08|11354400| 0.4800000000000004|        1.0|
|2013-02-20| AAL|14.17|13.33|14.26|13.15|14725200| 1.1099999999999994|        1.0|
|2013-02-21| AAL|13.62|13.37|13.95| 12.9|11922100|  1.049999999999999|        1.0|
|201

In [108]:
# Drop unnecessary columns
dataset = dataset.drop('Name')
dataset.show()

+----------+-----+-----+-----+-----+--------+-------------------+-----------+
|      date| open|close| high|  low|  volume|               diff|Stock_Abbrv|
+----------+-----+-----+-----+-----+--------+-------------------+-----------+
|2013-02-08|15.07|14.75|15.12|14.63| 8407500|0.48999999999999844|        1.0|
|2013-02-11|14.89|14.46|15.01|14.26| 8882000|               0.75|        1.0|
|2013-02-12|14.45|14.27|14.51| 14.1| 8126000|0.41000000000000014|        1.0|
|2013-02-13| 14.3|14.66|14.94|14.25|10259500| 0.6899999999999995|        1.0|
|2013-02-14|14.94|13.99|14.96|13.16|31879900| 1.8000000000000007|        1.0|
|2013-02-15|13.93| 14.5|14.61|13.93|15628000| 0.6799999999999997|        1.0|
|2013-02-19|14.33|14.26|14.56|14.08|11354400| 0.4800000000000004|        1.0|
|2013-02-20|14.17|13.33|14.26|13.15|14725200| 1.1099999999999994|        1.0|
|2013-02-21|13.62|13.37|13.95| 12.9|11922100|  1.049999999999999|        1.0|
|2013-02-22|13.57|13.57| 13.6|13.21| 6071400| 0.3899999999999988

In [96]:
dataset.dtypes

[('date', 'date'),
 ('open', 'double'),
 ('close', 'double'),
 ('high', 'double'),
 ('low', 'double'),
 ('volume', 'int'),
 ('diff', 'double'),
 ('Stock_Abbrv', 'double')]

Converting the date column in to a numeric encoded entry 

In [109]:
dataset=dataset.withColumn("dateAsInteger", F.unix_timestamp(dataset['date']))

In [110]:
dataset = dataset.drop('date')
dataset.show()

+-----+-----+-----+-----+--------+-------------------+-----------+-------------+
| open|close| high|  low|  volume|               diff|Stock_Abbrv|dateAsInteger|
+-----+-----+-----+-----+--------+-------------------+-----------+-------------+
|15.07|14.75|15.12|14.63| 8407500|0.48999999999999844|        1.0|   1360299600|
|14.89|14.46|15.01|14.26| 8882000|               0.75|        1.0|   1360558800|
|14.45|14.27|14.51| 14.1| 8126000|0.41000000000000014|        1.0|   1360645200|
| 14.3|14.66|14.94|14.25|10259500| 0.6899999999999995|        1.0|   1360731600|
|14.94|13.99|14.96|13.16|31879900| 1.8000000000000007|        1.0|   1360818000|
|13.93| 14.5|14.61|13.93|15628000| 0.6799999999999997|        1.0|   1360904400|
|14.33|14.26|14.56|14.08|11354400| 0.4800000000000004|        1.0|   1361250000|
|14.17|13.33|14.26|13.15|14725200| 1.1099999999999994|        1.0|   1361336400|
|13.62|13.37|13.95| 12.9|11922100|  1.049999999999999|        1.0|   1361422800|
|13.57|13.57| 13.6|13.21| 60

Dropping NAN values

In [115]:
dataset = dataset.na.drop()
dataset.show()

+-----+-----+-----+-----+--------+-------------------+-----------+-------------+
| open|close| high|  low|  volume|               diff|Stock_Abbrv|dateAsInteger|
+-----+-----+-----+-----+--------+-------------------+-----------+-------------+
|15.07|14.75|15.12|14.63| 8407500|0.48999999999999844|        1.0|   1360299600|
|14.89|14.46|15.01|14.26| 8882000|               0.75|        1.0|   1360558800|
|14.45|14.27|14.51| 14.1| 8126000|0.41000000000000014|        1.0|   1360645200|
| 14.3|14.66|14.94|14.25|10259500| 0.6899999999999995|        1.0|   1360731600|
|14.94|13.99|14.96|13.16|31879900| 1.8000000000000007|        1.0|   1360818000|
|13.93| 14.5|14.61|13.93|15628000| 0.6799999999999997|        1.0|   1360904400|
|14.33|14.26|14.56|14.08|11354400| 0.4800000000000004|        1.0|   1361250000|
|14.17|13.33|14.26|13.15|14725200| 1.1099999999999994|        1.0|   1361336400|
|13.62|13.37|13.95| 12.9|11922100|  1.049999999999999|        1.0|   1361422800|
|13.57|13.57| 13.6|13.21| 60

In [116]:
dataset.select([count(when(isnan(c), c)).alias(c) for c in dataset.columns]).show()

+----+-----+----+---+------+----+-----------+-------------+
|open|close|high|low|volume|diff|Stock_Abbrv|dateAsInteger|
+----+-----+----+---+------+----+-----------+-------------+
|   0|    0|   0|  0|     0|   0|          0|            0|
+----+-----+----+---+------+----+-----------+-------------+



In [146]:
dataset = dataset.drop('Stock_Abbrv')

In [147]:
dataset.show()

+-----+-----+-----+-----+--------+-------------------+-------------+
| open|close| high|  low|  volume|               diff|dateAsInteger|
+-----+-----+-----+-----+--------+-------------------+-------------+
|15.07|14.75|15.12|14.63| 8407500|0.48999999999999844|   1360299600|
|14.89|14.46|15.01|14.26| 8882000|               0.75|   1360558800|
|14.45|14.27|14.51| 14.1| 8126000|0.41000000000000014|   1360645200|
| 14.3|14.66|14.94|14.25|10259500| 0.6899999999999995|   1360731600|
|14.94|13.99|14.96|13.16|31879900| 1.8000000000000007|   1360818000|
|13.93| 14.5|14.61|13.93|15628000| 0.6799999999999997|   1360904400|
|14.33|14.26|14.56|14.08|11354400| 0.4800000000000004|   1361250000|
|14.17|13.33|14.26|13.15|14725200| 1.1099999999999994|   1361336400|
|13.62|13.37|13.95| 12.9|11922100|  1.049999999999999|   1361422800|
|13.57|13.57| 13.6|13.21| 6071400| 0.3899999999999988|   1361509200|
| 13.6|13.02|13.76| 13.0| 7186400| 0.7599999999999998|   1361768400|
|13.14|13.26|13.42| 12.7| 9419000|

Converting the variables into a single target variable and scaling them using Min Max Scaler

In [198]:
vecAssembler = VectorAssembler(inputCols=dataset.columns, outputCol="features",handleInvalid='skip')
normalizer = MinMaxScaler(inputCol="features", outputCol="scaledFeatures",min=0,max=1)
pipeline_normalize = Pipeline(stages=[vecAssembler,normalizer])
df_transf = pipeline_normalize.fit(dataset).transform(dataset)
#%%
to_array = F.udf(lambda x: (x.toArray().tolist()), ArrayType(DoubleType())) # IF spark 3.0.0 then a api - vectortoarray available
df_array = df_transf.withColumn("scaledFeatures",to_array("scaledFeatures"))
#%%\
df_final= df_array.select([F.col('scaledFeatures')[i].alias(dataset.columns[i]) for i in range(len(dataset.columns))])

In [199]:
df_final.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                open|               close|                high|                 low|              volume|                diff|       dateAsInteger|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|0.006585454224972825|0.006427632960667379|0.006499540241010503|0.006456498542001663|  0.0135989787187442| 0.00537847886510487|                 0.0|
|0.006497321752073561|0.006285990592993098|0.006446304989594929|0.006274556084991715|0.014366483080324295|0.007255531891852887|0.001643835616438356|
|0.006281886818319802|0.006193190421068569|0.006204326574069594| 0.00619587826574417|0.013143652105920604|0.004800924087643957|0.002191780821917808|
|0.006208443090903749|0.006383674984492603|0.006412428011421...|0.006269638721288743|0.016594591105775464|

**Converting to Feature vectors and applying minmax scaling**

In [215]:
required_features = ['dateAsInteger',
                    'open',
                    'close',
                    'volume',
                    'low',
                    'diff',
                   ]
assembler = VectorAssembler(inputCols=required_features, outputCol='features')
transformed_data = assembler.transform(df_final)

In [216]:
transformed_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                open|               close|                high|                 low|              volume|                diff|       dateAsInteger|            features|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|0.006585454224972825|0.006427632960667379|0.006499540241010503|0.006456498542001663|  0.0135989787187442| 0.00537847886510487|                 0.0|[0.0,0.0065854542...|
|0.006497321752073561|0.006285990592993098|0.006446304989594929|0.006274556084991715|0.014366483080324295|0.007255531891852887|0.001643835616438356|[0.00164383561643...|
|0.006281886818319802|0.006193190421068569|0.006204326574069594| 0.00619587826574417|0.013143652105920604|0.004800924087643957|0.002191780821917808|[0

**Modeling**

In [217]:
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(transformed_data)

In [218]:
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2])

In [219]:
training_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+
|                open|               close|                high|                 low|              volume|                diff|      dateAsInteger|            features|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+
|2.448124247201785E-5|1.465265872492565E-5|4.839568310506708E-6|5.409100073268724E-5|0.019274738981431196|0.002490704977800256| 0.4925799086757991|[0.49257990867579...|
|2.448124247201785E-5|3.907375659980162...|                 0.0| 7.37604555445734E-5|0.011330955613986999|0.002129733241887179|  0.527648401826484|[0.52764840182648...|
|2.937749096642144...|6.349485447467771E-5|1.935827324202683E-5|7.867781924754497E-5|0.018193007496961577|0.002346316283435026| 0.5249086757990867|[0.52490

In [220]:
rf = RandomForestRegressor(featuresCol="indexedFeatures",labelCol='high')
pipeline = Pipeline(stages=[featureIndexer, rf])

In [221]:
model = pipeline.fit(training_data)

In [222]:
predictions = model.transform(test_data)

In [224]:
evaluator = RegressionEvaluator(
    labelCol="volume", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = ", rmse)
accuracy = evaluator.evaluate(predictions)
print("Accuracy on test data = s", accuracy)

Root Mean Squared Error (RMSE) on test data =  0.05810955446086183
Accuracy on test data =  0.8456334466473277
